In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from joblib import Parallel, delayed
import pennylane as qml
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

# Load the Breast Cancer dataset
cancer_data = load_breast_cancer()
features_cancer = cancer_data.data
labels_cancer = cancer_data.target

# Use only the first two features for visualization purposes
features_cancer_2d = features_cancer[:, :2]

# Select a smaller subset for computational efficiency
subset_indices_cancer = np.random.choice(len(features_cancer_2d), 40, replace=False)
features_cancer_2d = features_cancer_2d[subset_indices_cancer]
labels_cancer = labels_cancer[subset_indices_cancer]

# Define the number of qubits based on the reduced feature set (2 features)
num_qubits_cancer = features_cancer_2d.shape[1]
device_cancer = qml.device('default.qubit', wires=num_qubits_cancer)

# Define a simpler quantum feature map
def quantum_map_cancer(x, params):
    for i in range(num_qubits_cancer):
        qml.RY(x[i], wires=i)
    for i in range(num_qubits_cancer):
        qml.RZ(params[i], wires=i)

# Define the variational quantum circuit
@qml.qnode(device_cancer)
def variational_circuit_cancer(x, params):
    quantum_map_cancer(x, params)
    return qml.state()

# Function to compute the kernel matrix element
def kernel_element_cancer(x1, x2, params):
    state_x1 = variational_circuit_cancer(x1, params)
    state_x2 = variational_circuit_cancer(x2, params)
    return np.abs(np.dot(np.conj(state_x1), state_x2))**2

# Function to compute the QEK matrix in parallel
def qek_matrix_cancer(X, params):
    n_samples = len(X)
    qek_matrix = np.zeros((n_samples, n_samples))

    def compute_element(i, j):
        return kernel_element_cancer(X[i], X[j], params)

    results = Parallel(n_jobs=-1)(delayed(compute_element)(i, j) for i in range(n_samples) for j in range(n_samples))
    
    for idx, value in enumerate(results):
        i = idx // n_samples
        j = idx % n_samples
        qek_matrix[i, j] = value

    return qek_matrix

# Kernel-target alignment function
def alignment_cancer(params, X, y):
    qek_matrix = qek_matrix_cancer(X, params)
    y_matrix = np.outer(y, y)
    alignment = np.sum(qek_matrix * y_matrix)
    return -alignment  # Negate for minimization

# Initialize random parameters
np.random.seed(42)
initial_params_cancer = np.random.uniform(0, np.pi, num_qubits_cancer)

# Optimize the parameters
result_cancer = minimize(alignment_cancer, initial_params_cancer, args=(features_cancer_2d, labels_cancer), method='COBYLA')
optimized_params_cancer = result_cancer.x

# Compute the optimized QEK matrix
optimized_qek_matrix_cancer = qek_matrix_cancer(features_cancer_2d, optimized_params_cancer)

# Split the dataset into training and testing sets
features_train_cancer, features_test_cancer, labels_train_cancer, labels_test_cancer = train_test_split(features_cancer_2d, labels_cancer, test_size=0.2, random_state=42)

# Create a mesh to plot the decision boundary
h_cancer = .5  # Increase the step size to reduce computation time
x_min_cancer, x_max_cancer = features_cancer_2d[:, 0].min() - 1, features_cancer_2d[:, 0].max() + 1
y_min_cancer, y_max_cancer = features_cancer_2d[:, 1].min() - 1, features_cancer_2d[:, 1].max() + 1
xx_cancer, yy_cancer = np.meshgrid(np.arange(x_min_cancer, x_max_cancer, h_cancer), np.arange(y_min_cancer, y_max_cancer, h_cancer))
grid_points_cancer = np.c_[xx_cancer.ravel(), yy_cancer.ravel()]

# Precompute the kernel matrix between the grid points and training data
def grid_kernel_matrix_cancer(grid_points, X_train, params):
    n_grid = len(grid_points)
    n_train = len(X_train)
    kernel_matrix = np.zeros((n_grid, n_train))

    def compute_element(i, j):
        return kernel_element_cancer(grid_points[i], X_train[j], params)

    results = Parallel(n_jobs=-1)(delayed(compute_element)(i, j) for i in range(n_grid) for j in range(n_train))

    for idx, value in enumerate(results):
        i = idx // n_train
        j = idx % n_train
        kernel_matrix[i, j] = value

    return kernel_matrix

# Compute the kernel matrix for the grid points
grid_kernel_matrix_cancer = grid_kernel_matrix_cancer(grid_points_cancer, features_train_cancer, optimized_params_cancer)

# Compute the optimized kernel matrix for the reduced 2D training set
optimized_qek_matrix_train_cancer = qek_matrix_cancer(features_train_cancer, optimized_params_cancer)

# Train the SVM classifier on the 2D quantum kernel
svm_cancer = SVC(kernel='precomputed')
svm_cancer.fit(optimized_qek_matrix_train_cancer, labels_train_cancer)

# Predict the class for each point in the mesh using the precomputed kernel matrix
Z_cancer = svm_cancer.predict(grid_kernel_matrix_cancer)
Z_cancer = Z_cancer.reshape(xx_cancer.shape)

# Plot the decision boundary by assigning a color to each point in the mesh
plt.contourf(xx_cancer, yy_cancer, Z_cancer, cmap=plt.cm.coolwarm, alpha=0.8)

# Plot also the training points
plt.scatter(features_cancer_2d[:, 0], features_cancer_2d[:, 1], c=labels_cancer, cmap=plt.cm.coolwarm, edgecolors='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('SVM Decision Boundary with Quantum Kernel (Breast Cancer Dataset)')
plt.show()

# Evaluate the classifier's accuracy on the test set
accuracy_cancer = svm_cancer.score(optimized_qek_matrix_train_cancer, labels_train_cancer)
print(f'Classification accuracy: {accuracy_cancer:.2f}')
